In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
import re

In [ ]:
df = pd.read_csv(r'C:\Users\e.katsadaki\Desktop\test_climate\IPCC_AR6_WGII_Chapter16.csv')
df

In [ ]:

# Drop 8 columns using the drop() method
df = df.drop(['Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9'], axis=1)

# Rename one column using the rename() method
df = df.rename(columns={'Unnamed: 0': 'text'})


In [ ]:
df= df.dropna()
df

In [ ]:
# Define the stopwords and other words to remove
#stop_words = set(stopwords.words('english'))
common_words = set(['and', 'as', 'or', 'that', 'he', 'she','be','they','them','yes','no','that', 'there','very','are', 'is'])
#remove_words = stop_words.union(common_words)

# Remove stopwords and other words from the input text
def preprocess_text(text):
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Tokenize the text
    tokens = text.split()
    # Remove stopwords and common words
    filtered_tokens = [token for token in tokens if token.lower() not in common_words]
    # Join the filtered tokens back into a string
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

df['text'] = df['text'].apply(preprocess_text)

In [ ]:
df['text'] = df['text'].str.replace('\d+', '') # remove numbers
df['text'] = df['text'].str.replace(r'\b(?:and|he|she|that|as|or|yes|no|be|by|in|into|as|there|are|is|very|they|the)\b\s*', '') # remove stop words and other unwanted words

In [ ]:
# Extract concepts from the input file
count_vect = CountVectorizer(ngram_range=(2, 3)) # extract 2-3 word concepts
X_train_counts = count_vect.fit_transform(df['text'])
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
feature_names = count_vect.get_feature_names()
sum_tfidf = X_train_tfidf.sum(axis=0)
sum_tfidf_list = [(feature_names[i], sum_tfidf[0,i]) for i in range(sum_tfidf.shape[1])]
sorted_tfidf_list = sorted(sum_tfidf_list, key=lambda x: x[1], reverse=True)

In [ ]:
# Convert the text into a bag of words
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(df['text'])

# Convert the bag of words into TF-IDF representation
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# Get the feature names
feature_names = count_vect.get_feature_names()

# Sum the TF-IDF scores for each feature across all documents
sum_tfidf = X_train_tfidf.sum(axis=0)

# Convert the sums into a list of tuples, where each tuple contains a feature name and its sum
sum_tfidf_list = [(feature_names[i], sum_tfidf[0,i]) for i in range(sum_tfidf.shape[1])]

# Sort the list of tuples by the second element in descending order
sorted_tfidf_list = sorted(sum_tfidf_list, key=lambda x: x[1], reverse=True)


In [ ]:
# Convert the sorted list of tuples into a dataframe
df_concepts = pd.DataFrame(sorted_tfidf_list, columns=['Concept', 'Importance'])

# Save the dataframe to an Excel file
df_concepts.to_excel(r'C:\Users\e.katsadaki\Desktop\test_climate\concepts_tf_idf.xlsx', index=False)
